In [1]:
import os
import tensorflow as tf
from transformers import BertTokenizer, TFBertForSequenceClassification
from tensorflow.keras.optimizers import Adam
from sklearn.model_selection import train_test_split
import numpy as np
from sklearn.metrics import confusion_matrix
import seaborn as sns
import matplotlib.pyplot as plt

/opt/anaconda3/envs/myenv6/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
class CustomTFBertForSequenceClassification(TFBertForSequenceClassification):
    def train_step(self, data):
        # Unpack the data manually
        x, y, sample_weight = self.unpack_data(data)

        with tf.GradientTape() as tape:
            y_pred = self(x, training=True)  # Forward pass
            loss = self.compiled_loss(y, y_pred, sample_weight, regularization_losses=self.losses)

        # Backward pass and optimization
        trainable_vars = self.trainable_variables
        gradients = tape.gradient(loss, trainable_vars)
        self.optimizer.apply_gradients(zip(gradients, trainable_vars))

        # Update the metrics
        self.compiled_metrics.update_state(y, y_pred, sample_weight)
        
        return {m.name: m.result() for m in self.metrics}

    def test_step(self, data):
        # Unpack the data manually
        x, y, sample_weight = self.unpack_data(data)

        # Forward pass
        y_pred = self(x, training=False)
        # Compute loss
        loss = self.compiled_loss(y, y_pred, sample_weight, regularization_losses=self.losses)

        # Update the metrics
        self.compiled_metrics.update_state(y, y_pred, sample_weight)

        return {m.name: m.result() for m in self.metrics}

    def unpack_data(self, data):
        if len(data) == 2:
            return data[0], data[1], None  # inputs, labels, sample_weights
        elif len(data) == 3:
            return data  # inputs, labels, sample_weights
        else:
            raise ValueError("Unexpected number of elements in `data`")

In [3]:
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = CustomTFBertForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=3)

/opt/anaconda3/envs/myenv6/lib/python3.11/site-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(
All PyTorch model weights were used when initializing CustomTFBertForSequenceClassification.

Some weights or buffers of the TF 2.0 model CustomTFBertForSequenceClassification were not initialized from the PyTorch model and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [4]:
# Function to load texts and labels from subdirectories
def load_texts_and_labels_from_directories(base_directory):
    texts = []
    labels = []
    label_map = {}
    label_index = 0
    
    for label in os.listdir(base_directory):
        label_dir = os.path.join(base_directory, label)
        if os.path.isdir(label_dir):
            if label not in label_map:
                label_map[label] = label_index
                label_index += 1
            for filename in os.listdir(label_dir):
                filepath = os.path.join(label_dir, filename)
                if os.path.isfile(filepath) and filename.endswith('.txt'):
                    with open(filepath, 'r', encoding='utf-8') as file:
                        texts.append(file.read())
                        labels.append(label_map[label])
    return texts, labels, label_map


In [5]:
def tokenize_texts(texts, tokenizer, max_len=128):
    input_ids = []
    attention_masks = []
    for text in texts:
        encoded_dict = tokenizer.encode_plus(
            text,
            add_special_tokens=True,
            max_length=max_len,
            padding='max_length',
            return_attention_mask=True,
            truncation=True,
            return_tensors='tf'
        )
        input_ids.append(encoded_dict['input_ids'])
        attention_masks.append(encoded_dict['attention_mask'])
    return tf.concat(input_ids, axis=0), tf.concat(attention_masks, axis=0)

In [6]:
from sklearn.model_selection import StratifiedShuffleSplit

# Base directory where your data is stored
base_directory = 'data_v1'  # Replace with the actual path
texts, labels, label_map = load_texts_and_labels_from_directories(base_directory)

# Tokenize texts
input_ids, attention_masks = tokenize_texts(texts, tokenizer)

# Convert labels into tensors
labels = tf.convert_to_tensor(labels)

# Convert TensorFlow tensors to NumPy arrays for splitting
input_ids_np = input_ids.numpy()
attention_masks_np = attention_masks.numpy()
labels_np = labels.numpy()

# Use StratifiedShuffleSplit to maintain category distribution
sss = StratifiedShuffleSplit(n_splits=1, test_size=0.3, random_state=42)

for train_index, temp_index in sss.split(input_ids_np, labels_np):
    train_inputs, temp_inputs = input_ids_np[train_index], input_ids_np[temp_index]
    train_labels, temp_labels = labels_np[train_index], labels_np[temp_index]
    train_masks, temp_masks = attention_masks_np[train_index], attention_masks_np[temp_index]

# Now split the temp set into validation and test sets, stratifying again
sss_val_test = StratifiedShuffleSplit(n_splits=1, test_size=0.5, random_state=42)

for val_index, test_index in sss_val_test.split(temp_inputs, temp_labels):
    validation_inputs, test_inputs = temp_inputs[val_index], temp_inputs[test_index]
    validation_labels, test_labels = temp_labels[val_index], temp_labels[test_index]
    validation_masks, test_masks = temp_masks[val_index], temp_masks[test_index]

# Convert everything back to tensors if needed for the model input
train_inputs = tf.convert_to_tensor(train_inputs)
validation_inputs = tf.convert_to_tensor(validation_inputs)
test_inputs = tf.convert_to_tensor(test_inputs)

train_masks = tf.convert_to_tensor(train_masks)
validation_masks = tf.convert_to_tensor(validation_masks)
test_masks = tf.convert_to_tensor(test_masks)

train_labels = tf.convert_to_tensor(train_labels)
validation_labels = tf.convert_to_tensor(validation_labels)
test_labels = tf.convert_to_tensor(test_labels)

In [7]:
print(validation_masks.shape)
print(train_masks.shape)
print(test_masks.shape)

(22, 128)
(105, 128)
(23, 128)


In [8]:
# Define the optimizer, loss function, and metrics
optimizer = Adam(learning_rate=2e-5, epsilon=1e-8)
loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)
metric = tf.keras.metrics.SparseCategoricalAccuracy('accuracy')

In [9]:
model.compile(optimizer=optimizer, loss=loss, metrics=[metric])

# Train the model
history = model.fit(
    [train_inputs, train_masks],
    train_labels,
    validation_data=([validation_inputs, validation_masks], validation_labels),
    epochs=4,
    batch_size=4
)

Epoch 1/4


2024-10-22 12:57:46.882144: W tensorflow/tsl/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz


27/27 [==============================] - 40s 1s/step - loss: 0.7971 - accuracy: 0.6381 - val_loss: 0.4295 - val_accuracy: 1.0000
Epoch 2/4
27/27 [==============================] - 31s 1s/step - loss: 0.2735 - accuracy: 0.9714 - val_loss: 0.1127 - val_accuracy: 1.0000
Epoch 3/4
27/27 [==============================] - 29s 1s/step - loss: 0.0916 - accuracy: 1.0000 - val_loss: 0.0451 - val_accuracy: 1.0000
Epoch 4/4
27/27 [==============================] - 30s 1s/step - loss: 0.0413 - accuracy: 1.0000 - val_loss: 0.0258 - val_accuracy: 1.0000


In [10]:
# Evaluate the model on the test set
test_loss, test_accuracy = model.evaluate([test_inputs, test_masks], test_labels)
print(f"Test Accuracy: {test_accuracy}")

1/1 [==============================] - 2s 2s/step - loss: 0.0257 - accuracy: 1.0000
Test Accuracy: 1.0


In [11]:
# Making predictions on the test set (optional)
predictions = model.predict([test_inputs, test_masks])
predicted_labels = tf.argmax(predictions.logits, axis=-1)
print("Predicted labels on the test set:", predicted_labels.numpy())

1/1 [==============================] - 3s 3s/step
Predicted labels on the test set: [0 2 2 1 1 2 1 1 1 1 1 1 1 2 0 1 1 1 0 1 1 0 1]


In [12]:
predicted_labels_np = predicted_labels.numpy()
true_labels_np = test_labels

# Compute the confusion matrix
conf_matrix = confusion_matrix(true_labels_np, predicted_labels_np)

print("Confusion Matrix:")
for row in conf_matrix:
    print(' '.join(map(str, row)))

# Optionally print the labels for reference
print("\nLabel mapping (index -> label name):")
for label_name, index in label_map.items():
    print(f"{index}: {label_name}")

Confusion Matrix:
4 0 0
0 15 0
0 0 4

Label mapping (index -> label name):
0: eraser
1: neutral
2: keys
